In [64]:
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
import torch

class CustomDataset(Dataset):
    def __init__(self, data_array, labels_array):
        self.data_array = data_array
        self.labels_array = labels_array

    def __len__(self):
        return len(self.data_array)

    def __getitem__(self, idx):
        return torch.Tensor(self.data_array[idx]), torch.Tensor([self.labels_array[idx]])

def collate_fn(batch):
    data, labels = zip(*batch)
    return torch.stack(data), torch.cat(labels)

# Your data processing code here
data_root = "train_data"
data_list = []
labels_list = []
max_sequence_length = 0

for language_folder in os.listdir(data_root):
    language_path = os.path.join(data_root, language_folder)

    if os.path.isdir(language_path):
        for npy_file in os.listdir(language_path):
            if npy_file.endswith(".npy"):
                npy_path = os.path.join(language_path, npy_file)
                data = np.load(npy_path)
                max_sequence_length = max(max_sequence_length, data.shape[0])
                data_list.append(data)
                labels_list.append(int(language_folder.split('_')[-1]))

test_root = "test_data"
test_list = []
for npy_file in os.listdir(test_root):
    if npy_file.endswith(".npy"):
            npy_path = os.path.join(test_root, npy_file)
            test = np.load(npy_path)
            max_sequence_length = max(max_sequence_length, test.shape[0])
            test_list.append(test)

for i in range(len(data_list)):
    current_sequence_length = data_list[i].shape[0]
    if current_sequence_length < max_sequence_length:
        pad_length = max_sequence_length - current_sequence_length
        data_list[i] = np.pad(data_list[i], ((0, pad_length), (0, 0)), 'wrap')

for i in range(len(test_list)):
    current_sequence_length = test_list[i].shape[0]
    if current_sequence_length < max_sequence_length:
        pad_length = max_sequence_length - current_sequence_length
        test_list[i] = np.pad(test_list[i], ((0, pad_length), (0, 0)), 'wrap')

data_array = np.array(data_list)
labels_array = np.array(labels_list)

test_array = np.array(test_list)
test_tensor = torch.tensor(test_array)

# Create a dataset
dataset = CustomDataset(data_array, labels_array)

# Split the dataset into training and validation sets
total_samples = len(dataset)
train_size = int(0.8 * total_samples)
val_size = total_samples - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, drop_last=False)

test_loader = DataLoader(test_tensor, batch_size=batch_size, shuffle=False,drop_last=False)

In [88]:
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_size = 80  # Assuming 80 features in your data
sequence_length = max_sequence_length
hidden_size = 128
num_classes = 2


class TransformerClassifier(nn.Module):
    def __init__(self, hidden_size, num_classes, num_heads=4, num_encoder_layers=10, dropout=0.3):
        super(TransformerClassifier, self).__init__()

        out_channels = 256
        kernel_size = 5
        self.conv1d = nn.Conv1d(in_channels=sequence_length, out_channels=out_channels, kernel_size=kernel_size)
        self.relu = nn.SELU()
        self.embedding = nn.Linear(out_channels * (feature_size - kernel_size + 1), hidden_size)

        self.transformer_encoder_layers = nn.TransformerEncoderLayer(
            d_model=hidden_size, nhead=num_heads, dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(
            self.transformer_encoder_layers, num_layers=num_encoder_layers
        )

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.relu(self.conv1d(x))
        x = x.view(x.shape[0], -1)
        x = self.embedding(x)
        x = x.view(-1, x.shape[0], x.shape[1])
        output = self.transformer_encoder(x)
        output = output.view(output.shape[1], -1)
        output = self.fc(output)

        return output

# Move the model to GPU
model = TransformerClassifier(hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    for batch_data, batch_labels in train_loader:
        # Move data to GPU
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)

        # Forward pass
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels.long())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for val_data, val_labels in val_loader:
            # Move validation data to GPU
            val_data, val_labels = val_data.to(device), val_labels.to(device)

            val_outputs = model(val_data)
            _, predicted = torch.max(val_outputs.data, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels.long()).sum().item()

        accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {100 * accuracy:.2f}%')

d:\anaconda\envs\big_data\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])
torch.Size([1, 32, 128])


KeyboardInterrupt: 

In [1]:
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
import torch

class CustomDataset(Dataset):
    def __init__(self, data_array, labels_array):
        self.data_array = data_array
        self.labels_array = labels_array

    def __len__(self):
        return len(self.data_array)

    def __getitem__(self, idx):
        return torch.Tensor(self.data_array[idx]), torch.Tensor([self.labels_array[idx]])

def collate_fn(batch):
    data, labels = zip(*batch)
    return torch.stack(data), torch.cat(labels)

# Your data processing code here
data_root = "train_data"
data_list = []
labels_list = []
max_sequence_length = 0
min_sequence_length = 10000

for language_folder in os.listdir(data_root):
    language_path = os.path.join(data_root, language_folder)

    if os.path.isdir(language_path):
        for npy_file in os.listdir(language_path):
            if npy_file.endswith(".npy"):
                npy_path = os.path.join(language_path, npy_file)
                data = np.load(npy_path)
                max_sequence_length = max(max_sequence_length, data.shape[0])
                min_sequence_length = min(min_sequence_length, data.shape[0])
                data_list.append(data)
                labels_list.append(int(language_folder.split('_')[-1]))

test_root = "test_data"
test_list = []
for npy_file in os.listdir(test_root):
    if npy_file.endswith(".npy"):
            npy_path = os.path.join(test_root, npy_file)
            test = np.load(npy_path)
            max_sequence_length = max(max_sequence_length, test.shape[0])
            min_sequence_length = min(min_sequence_length, test.shape[0])
            test_list.append(test)

for i in range(len(data_list)):
    current_sequence_length = data_list[i].shape[0]
    if current_sequence_length > min_sequence_length:
        data_list[i] = data_list[i][:min_sequence_length, :]

for i in range(len(test_list)):
    current_sequence_length = test_list[i].shape[0]
    if current_sequence_length > min_sequence_length:
        test_list[i] = test_list[i][:min_sequence_length,:]

data_array = np.array(data_list)
labels_array = np.array(labels_list)

test_array = np.array(test_list)
test_tensor = torch.tensor(test_array)

# Create a dataset
dataset = CustomDataset(data_array, labels_array)

# Split the dataset into training and validation sets
total_samples = len(dataset)
train_size = int(0.8 * total_samples)
val_size = total_samples - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, drop_last=False)

test_loader = DataLoader(test_tensor, batch_size=batch_size, shuffle=False,drop_last=False)

In [15]:
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_size = 80  # Assuming 80 features in your data
sequence_length = min_sequence_length


model_params = {
        'hidden_size': 128,
        'num_heads': 4,
        'num_encoder_layers': 4,
        'dropout': 0.3,
        'out_channels': 32,
        'kernel_size': 3
    }


class TransformerClassifier(nn.Module):
    def __init__(self, hidden_size=128, num_heads=4, num_encoder_layers=4, dropout=0.3, out_channels=32, kernel_size=3):
        super(TransformerClassifier, self).__init__()

        self.conv1d = nn.Conv1d(in_channels=sequence_length, out_channels=out_channels, kernel_size=kernel_size)
        self.relu = nn.SELU()
        self.embedding = nn.Linear(out_channels * (feature_size - kernel_size + 1), hidden_size)

        self.transformer_encoder_layers = nn.TransformerEncoderLayer(
            d_model=hidden_size, nhead=num_heads, dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(
            self.transformer_encoder_layers, num_layers=num_encoder_layers
        )

        self.fc = nn.Linear(hidden_size, 2)

    def forward(self, x):
        x = self.relu(self.conv1d(x))
        x = x.view(x.shape[0], -1)
        x = self.embedding(x)
        x = x.view(-1, x.shape[0], x.shape[1])
        output = self.transformer_encoder(x)
        output = output.view(output.shape[1], -1)
        output = self.fc(output)

        return output

# Move the model to GPU
model = TransformerClassifier().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 100
model.train()
for epoch in range(num_epochs):
    for batch_data, batch_labels in train_loader:
        # Move data to GPU
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)

        # Forward pass
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels.long())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for val_data, val_labels in val_loader:
            # Move validation data to GPU
            val_data, val_labels = val_data.to(device), val_labels.to(device)

            val_outputs = model(val_data)
            _, predicted = torch.max(val_outputs.data, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels.long()).sum().item()

        accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {100 * accuracy:.2f}%')
    model.train()

d:\anaconda\envs\big_data\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/100], Loss: 0.6843, Validation Accuracy: 69.38%
Epoch [2/100], Loss: 0.4639, Validation Accuracy: 70.75%
Epoch [3/100], Loss: 0.6886, Validation Accuracy: 74.38%
Epoch [4/100], Loss: 0.5892, Validation Accuracy: 81.00%
Epoch [5/100], Loss: 0.2976, Validation Accuracy: 84.88%
Epoch [6/100], Loss: 0.3876, Validation Accuracy: 86.12%
Epoch [7/100], Loss: 0.1351, Validation Accuracy: 88.12%
Epoch [8/100], Loss: 0.4037, Validation Accuracy: 89.12%
Epoch [9/100], Loss: 0.5056, Validation Accuracy: 89.25%
Epoch [10/100], Loss: 0.1265, Validation Accuracy: 90.12%
Epoch [11/100], Loss: 0.1735, Validation Accuracy: 90.25%
Epoch [12/100], Loss: 0.1958, Validation Accuracy: 91.00%
Epoch [13/100], Loss: 0.3312, Validation Accuracy: 90.50%
Epoch [14/100], Loss: 0.1755, Validation Accuracy: 90.25%
Epoch [15/100], Loss: 0.3453, Validation Accuracy: 90.75%
Epoch [16/100], Loss: 0.1386, Validation Accuracy: 88.12%
Epoch [17/100], Loss: 0.1494, Validation Accuracy: 89.88%
Epoch [18/100], Loss: 0

In [16]:
torch.save(model.state_dict(), 'save_models/'+'model_transformer_v1.pth')

In [18]:
test_label = []
model = TransformerClassifier().to(device)
model.load_state_dict(torch.load('save_models/model_transformer_v1.pth'))
model.eval()
with torch.no_grad():
    for test_data in test_loader:
        # Move validation data to GPU
        test_data = test_data.to(device)

        test_outputs = model(test_data)
        _, predicted = torch.max(test_outputs.data, 1)
        
        test_label.extend(predicted.tolist())

d:\anaconda\envs\big_data\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [19]:
sum(test_label)

927

In [29]:
import optuna

def objective(trial):

    hidden_size = trial.suggest_int('hidden_size', 64, 256)
    num_heads = trial.suggest_int('num_heads', 2, 8)
    num_encoder_layers = trial.suggest_int('num_encoder_layers', 2, 8)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    out_channels = trial.suggest_int('out_channels', 16, 64)
    kernel_size = trial.suggest_int('kernel_size', 3, 7)

    if hidden_size % num_heads != 0:
        hidden_size = (hidden_size // num_heads) * num_heads

    model = TransformerClassifier(
        hidden_size=hidden_size,
        num_heads=num_heads,
        num_encoder_layers=num_encoder_layers,
        dropout=dropout,
        out_channels=out_channels,
        kernel_size=kernel_size
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

    num_epochs = 100

    model.train()
    for epoch in range(num_epochs):
        for batch_data, batch_labels in train_loader:
            # Move data to GPU
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)

            # Forward pass
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels.long())

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for val_data, val_labels in val_loader:
            # Move validation data to GPU
            val_data, val_labels = val_data.to(device), val_labels.to(device)

            val_outputs = model(val_data)
            _, predicted = torch.max(val_outputs.data, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels.long()).sum().item()

        accuracy = correct / total
        print(f'{hidden_size}_{num_heads}_{num_encoder_layers}_{dropout}_{out_channels}_{kernel_size}_Validation Accuracy: {100 * accuracy:.2f}%')
    
    torch.save(model.state_dict(), 'save_models/'+f'model_transformer_{hidden_size}_{num_heads}_{num_encoder_layers}_{dropout}_{out_channels}_{kernel_size}.pth')
    
    return accuracy


study = optuna.create_study(direction='maximize') 
study.optimize(objective, n_trials=50)  

best_params = study.best_params
best_value = study.best_value
print(f"Best hyperparameters: {best_params}")
print(f"Best value (accuracy): {best_value}")


[I 2023-12-22 22:03:12,405] A new study created in memory with name: no-name-d76b1ec4-5c14-4054-bc79-86747e044d41
[I 2023-12-22 22:05:55,981] Trial 0 finished with value: 0.9375 and parameters: {'hidden_size': 66, 'num_heads': 3, 'num_encoder_layers': 5, 'dropout': 0.26792001323294157, 'out_channels': 60, 'kernel_size': 6}. Best is trial 0 with value: 0.9375.


66_3_5_0.26792001323294157_60_6_Validation Accuracy: 93.75%


[I 2023-12-22 22:09:35,110] Trial 1 finished with value: 0.9425 and parameters: {'hidden_size': 133, 'num_heads': 5, 'num_encoder_layers': 7, 'dropout': 0.34948548858777095, 'out_channels': 39, 'kernel_size': 6}. Best is trial 1 with value: 0.9425.


130_5_7_0.34948548858777095_39_6_Validation Accuracy: 94.25%


[I 2023-12-22 22:13:48,738] Trial 2 finished with value: 0.93625 and parameters: {'hidden_size': 201, 'num_heads': 3, 'num_encoder_layers': 7, 'dropout': 0.15005351812746826, 'out_channels': 21, 'kernel_size': 6}. Best is trial 1 with value: 0.9425.


...

160_8_4_0.1416134672882693_37_3_Validation Accuracy: 95.25%


[I 2023-12-23 00:35:00,735] Trial 48 finished with value: 0.93625 and parameters: {'hidden_size': 242, 'num_heads': 7, 'num_encoder_layers': 4, 'dropout': 0.14264517872164809, 'out_channels': 30, 'kernel_size': 3}. Best is trial 47 with value: 0.9525.


238_7_4_0.14264517872164809_30_3_Validation Accuracy: 93.62%


[I 2023-12-23 00:37:47,230] Trial 49 finished with value: 0.93875 and parameters: {'hidden_size': 189, 'num_heads': 6, 'num_encoder_layers': 4, 'dropout': 0.06634575697243457, 'out_channels': 33, 'kernel_size': 3}. Best is trial 47 with value: 0.9525.


186_6_4_0.06634575697243457_33_3_Validation Accuracy: 93.88%
Best hyperparameters: {'hidden_size': 161, 'num_heads': 8, 'num_encoder_layers': 4, 'dropout': 0.1416134672882693, 'out_channels': 37, 'kernel_size': 3}
Best value (accuracy): 0.9525
